In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import unidecode
from pattern.en import sentiment, polarity, subjectivity
%matplotlib inline

In [2]:
def sentiment(text):
    return polarity(text)

def subj(text):
    return subjectivity(text)

In [3]:
def remove_html_tags(data):
    p = re.compile(r'<[^<]*?>')
    return p.sub('', data)

In [4]:
def sentiment_new(text):
    def Sentences(paragraph):
        sentenceEnders = re.compile('[.!?]')
        sentenceList = sentenceEnders.split(paragraph)
        return sentenceList
    content = unidecode.unidecode(text).replace("\n"," ").replace("\'s","").replace("\'t","")
    s_max = 0
    comment_score = 0

    for i in Sentences(content):
        if abs(sentiment(i)) > abs(s_max):
            comment_score = sentiment(i)
            s_max = sentiment(i)
    return comment_score

In [5]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [6]:
def get_state(x):
    x_val = x.lower().title()
    if x_val in states.values():
        return states.keys()[states.values().index(x_val)]
    elif x in states.keys():
        return x
    else:
        return None

def state(loc):
    if get_state(loc) != None:
        return get_state(loc)
    else: 
        tokens = loc.upper().split(', ')
        if len(tokens) == 1:
            return get_state(tokens[0])
        else:
            return get_state(tokens[1])

def city(loc):
    tokens = loc.lower().title().split(', ')

    for obj in tokens:
        if obj in states.values():
            return None
        elif obj  == 'Usa' or tokens == 'Us':
            return None
        else: 
            return tokens[0]

In [7]:
def remove_link(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

In [8]:
longlat = pd.read_csv('data/longlat.csv')

In [9]:
candidates = ['hilary', 'sanders', 'biden', 'trump', 'bush', 'carson'] 

In [11]:
for c in candidates:
    data = pd.read_csv('data/'+c+'_meta.csv')
    data[pd.isnull(data['Comment'])] = ""
    data = data.drop_duplicates('Comment')
    data['Comment'] = data['Comment'].apply(lambda x: remove_link(str(x)))
    data['Comment'] = data.apply(lambda row: remove_html_tags(row['Comment']), axis = 1)
    data['Sentiment'] = data.apply(lambda row: sentiment_new(row['Comment']), axis = 1)
    data['Sentiment_raw'] = data.apply(lambda row: sentiment(row['Comment']), axis = 1)
    data['State'] = data.apply(lambda row: state(str(row['Locations'])), axis = 1)
    data['City'] = data.apply(lambda row: city(str(row['Locations'])), axis = 1)
    data = pd.merge(data, longlat, how='left', on='State')
    #data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace= True)

#    data['Sentiment_sent'] = data.apply(lambda row: sentiment_new(row['Comment']), axis = 1)
#     data['Sentiment_b'] = data['Sentiment'] >= 0
#     data['Sentiment_b'] = data['Sentiment_b'].astype(int) 
#     data['Sentiment_b_sent'] = data['Sentiment_sent'] >= 0
#     data['Sentiment_b_sent'] = data['Sentiment_b_sent'].astype(int)
    
#     data['Subjectivity'] = data.apply(lambda row: subj(row['Comment']), axis = 1)
#     data['Subjectivity_sent'] = data.apply(lambda row: sentiment_new(row['Comment']), axis = 1)
#     data['Subjectivity_b'] = data['Subjectivity'] >= 0
#     data['Subjectivity_b'] = data['Subjectivity_b'].astype(int) 
#     data['Subjectivity_b_sent'] = data['Subjectivity_sent'] >= 0
#     data['Subjectivity_b_sent'] = data['Subjectivity_b_sent'].astype(int) 
    data.to_csv('data/'+c+'_scores.csv')

/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.


In [12]:
hilary = pd.read_csv('data/hilary_scores.csv')
sanders = pd.read_csv('data/sanders_scores.csv') ###
biden = pd.read_csv('data/biden_scores.csv')
trump = pd.read_csv('data/trump_scores.csv')
bush = pd.read_csv('data/bush_scores.csv')
carson = pd.read_csv('data/carson_scores.csv') ###

In [14]:
hilary.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Comment', u'URL', u'Recommendations',
       u'Locations', u'EditorPick', u'userID', u'date', u'Sentiment',
       u'Sentiment_raw', u'State', u'City', u'latitude', u'longitude'],
      dtype='object')

In [13]:
print len(hilary)
print len(sanders)
print len(biden)
print len(trump)
print len(bush)
print len(carson)

24983
11958
27380
26513
24974
22032


In [38]:
hilary['Candidate'] = 'Hilary'
sanders['Candidate'] = 'Sanders'
biden['Candidate'] = 'Biden'
trump['Candidate'] = 'Trump'
bush['Candidate'] = 'Bush'
carson['Candidate'] = 'Carson'

In [39]:
master = pd.concat([hilary, sanders, biden, trump, bush, carson])

In [40]:
master = pd.concat([hilary, sanders, biden, trump, bush, carson])

master['EditorPick'] = master['EditorPick'].apply(lambda x: 0 if x == 'False' else x)
master['EditorPick'] = master['EditorPick'].apply(lambda x: 1 if x == 'True' else x)
master['EditorPick'] = master['EditorPick'].astype(float)
master['Recommendations'] = master['Recommendations'].apply(lambda x: 0 if type(x) == str else x)
master.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace= True)
master.to_csv('data/master.csv')

In [ ]:
''.join(ch for ch in string if ord(ch)<128)